In [86]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dphi-amsterdam-airbnb-data/airbnb_listing_validate.csv
/kaggle/input/dphi-amsterdam-airbnb-data/airbnb_listing_train.csv
/kaggle/input/dphi-amsterdam-airbnb-data/sample_submission.csv


In [87]:
airdata = pd.read_csv('/kaggle/input/dphi-amsterdam-airbnb-data/airbnb_listing_train.csv')

In [88]:
airdata.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price
0,11602914,Luxury home 100m2+garden in center!,3123809,Marjolein,NaN,De Pijp - Rivierenbuurt,52.35368,4.90177,Entire home/apt,3,13,15-02-2020,0.27,1,0,220
1,13289321,East is where your home is!,10259430,Ana,NaN,Oud-Oost,52.35728,4.92052,Entire home/apt,4,14,19-06-2019,0.29,1,0,110
2,40779315,Amsterdam Central - LUXURY house in BUSY STREET,224969266,Mark,NaN,Centrum-West,52.37582,4.89850,Entire home/apt,2,9,16-03-2020,1.65,1,7,100
3,7820311,great appartment down town,693472,Ayden,NaN,Westerpark,52.37792,4.87300,Entire home/apt,3,42,17-02-2020,0.72,1,0,130
4,27346603,Private room with canal view in peaceful area,41888346,Alissa,NaN,Westerpark,52.38051,4.87493,Private room,2,89,26-02-2020,4.02,1,24,90


In [89]:
airdata.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'price'],
      dtype='object')

### Cleaning of data begain 

In [90]:
airdata.isnull().sum()

id                                    0
name                                 22
host_id                               0
host_name                             8
neighbourhood_group               12901
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
minimum_nights                        0
number_of_reviews                     0
last_review                        1596
reviews_per_month                  1596
calculated_host_listings_count        0
availability_365                      0
price                                 0
dtype: int64

In [91]:
airdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12901 entries, 0 to 12900
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              12901 non-null  int64  
 1   name                            12879 non-null  object 
 2   host_id                         12901 non-null  int64  
 3   host_name                       12893 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   12901 non-null  object 
 6   latitude                        12901 non-null  float64
 7   longitude                       12901 non-null  float64
 8   room_type                       12901 non-null  object 
 9   minimum_nights                  12901 non-null  int64  
 10  number_of_reviews               12901 non-null  int64  
 11  last_review                     11305 non-null  object 
 12  reviews_per_month               

In [92]:
airdata['reviews_per_month'] = airdata['reviews_per_month'].fillna(np.mean(airdata['reviews_per_month']))

In [93]:
print(airdata.neighbourhood_group.unique())
print(airdata.neighbourhood.unique())
print(airdata.room_type.unique())

[nan]
['De Pijp - Rivierenbuurt' 'Oud-Oost' 'Centrum-West' 'Westerpark'
 'Centrum-Oost' 'De Baarsjes - Oud-West' 'Noord-West' 'Bos en Lommer'
 'De Aker - Nieuw Sloten' 'Oostelijk Havengebied - Indische Buurt'
 'Bijlmer-Centrum' 'Watergraafsmeer' 'Slotervaart' 'Osdorp' 'Zuid'
 'Geuzenveld - Slotermeer' 'Oud-Noord' 'Buitenveldert - Zuidas'
 'IJburg - Zeeburgereiland' 'Noord-Oost' 'Gaasperdam - Driemond'
 'Bijlmer-Oost']
['Entire home/apt' 'Private room' 'Shared room' 'Hotel room']


It only contain nan value so no need of this column

In [94]:
airdata = airdata.drop('neighbourhood_group',axis=1)


In [95]:
airdata['last_review']=airdata['last_review'].astype(str)
airdata['last_review'] = airdata['last_review'].fillna(airdata['last_review'].mode()[0])

In [96]:
airdata = airdata.dropna()

In [97]:
airdata.isnull().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
price                             0
dtype: int64

In [98]:
airdata.shape

(12871, 15)

# Exploratory Data analysis

In [100]:
airdata.corr()

,id,host_id,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price
id,1.000000,0.515241,0.006277,0.013464,-0.030058,-0.289309,0.029879,0.120599,0.045629,0.076865
host_id,0.515241,1.000000,0.001788,0.026581,-0.023978,-0.114314,0.091930,0.033921,0.121027,0.084473
latitude,0.006277,0.001788,1.000000,-0.154926,-0.005601,0.028970,0.024388,0.006429,0.034835,0.017939
longitude,0.013464,0.026581,-0.154926,1.000000,-0.001505,0.004364,0.017827,0.018871,0.035567,0.016594
minimum_nights,-0.030058,-0.023978,-0.005601,-0.001505,1.000000,-0.022013,-0.028391,-0.006957,0.056877,0.026531
number_of_reviews,-0.289309,-0.114314,0.028970,0.004364,-0.022013,1.000000,0.756355,-0.020717,0.273391,-0.035153
reviews_per_month,0.029879,0.091930,0.024388,0.017827,-0.028391,0.756355,1.000000,0.012834,0.281181,-0.002443
calculated_host_listings_count,0.120599,0.033921,0.006429,0.018871,-0.006957,-0.020717,0.012834,1.000000,0.095778,0.057298
availability_365,0.045629,0.121027,0.034835,0.035567,0.056877,0.273391,0.281181,0.095778,1.000000,0.154115
price,0.076865,0.084473,0.017939,0.016594,0.026531,-0.035153,-0.002443,0.057298,0.154115,1.000000
